In [1448]:
import pandas as pd

In [1449]:
# Load the pkl files
path = '../../data/processed/'
ct = pd.read_pickle(path + 'centrifuga_total.pkl')
cf = pd.read_pickle(path + 'cf.pkl')
h = pd.read_pickle(path + 'horas.pkl')
mc = pd.read_pickle(path + 'mov_componentes.pkl')
th = pd.read_pickle(path + 'th.pkl')
ino = pd.read_pickle(path + 'inoculo.pkl')
pino = pd.read_pickle(path + 'preinoculo.pkl')
of = pd.read_pickle(path + 'of.pkl')
bt = pd.read_pickle(path + 'biorreactor_total.pkl')


In [1450]:
# Check them all again
print(ct.info())
print(cf.info())
print(h.info())
print(mc.info())
print(ino.info())
print(pino.info())
print(th.info())
print(of.info())
print(bt.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157239 entries, 0 to 157238
Data columns (total 9 columns):
 #   Column                    Non-Null Count   Dtype              
---  ------                    --------------   -----              
 0   f_h_ct                    157239 non-null  datetime64[ns, UTC]
 1   desc_parcial_ct           154983 non-null  float64            
 2   desc_total_ct             155199 non-null  float64            
 3   apertura_valvula_agua_ct  156224 non-null  float64            
 4   caudal_ct                 156223 non-null  float64            
 5   contrapresion_ct          156223 non-null  float64            
 6   presion_agua_ct           111674 non-null  float64            
 7   vel_separacion_ct         156224 non-null  float64            
 8   id_centr                  157239 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(7), object(1)
memory usage: 10.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entrie

## Indexed by Lote

### cf + of

In [1451]:
# Lote in cf but not in of
cf[cf['lote'].isin(of['lote']) == False]

,lote,orden_encadenado_cf,lote_parental_cf,id_bio,f_h_inicio_cf,f_h_fin_cf,vol_ino_util_cf,turb_inicio_cultivo_cf,turb_fin_cultivo_cf,viab_final_cultivo_cf,id_centr,centr_1_turb_cf,centr_2_turb_cf,producto_1_cf,producto_2_cf,dur_cf,turbidez_diff_cf
143,24043,1,nan,14614,2024-03-12 06:25:00+00:00,2024-03-14 07:25:00+00:00,80.0,19.44,69.68,132800000.0,14246,30.32,16.16,1254.56,4.72,176400,50.24


Only 1 lote, which is good.

In [1452]:
# Merge cf and of on lote using left join
cf_of = pd.merge(cf, of, on='lote', how='left')
cf_of.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   lote                    152 non-null    object             
 1   orden_encadenado_cf     152 non-null    int64              
 2   lote_parental_cf        152 non-null    object             
 3   id_bio                  152 non-null    object             
 4   f_h_inicio_cf           152 non-null    datetime64[ns, UTC]
 5   f_h_fin_cf              152 non-null    datetime64[ns, UTC]
 6   vol_ino_util_cf         147 non-null    float64            
 7   turb_inicio_cultivo_cf  152 non-null    float64            
 8   turb_fin_cultivo_cf     152 non-null    float64            
 9   viab_final_cultivo_cf   152 non-null    float64            
 10  id_centr                152 non-null    object             
 11  centr_1_turb_cf         148 non-null    float

In [1453]:
cf_of.head()

,lote,orden_encadenado_cf,lote_parental_cf,id_bio,f_h_inicio_cf,f_h_fin_cf,vol_ino_util_cf,turb_inicio_cultivo_cf,turb_fin_cultivo_cf,viab_final_cultivo_cf,id_centr,centr_1_turb_cf,centr_2_turb_cf,producto_1_cf,producto_2_cf,dur_cf,turbidez_diff_cf,orden,cantidad_of
0,23019,1,nan,14615,2023-03-21 06:30:00+00:00,2023-03-23 05:30:00+00:00,82.4,17.28,91.2,184000000.0,17825,NaN,NaN,1747.920,6.00,169200,73.92,200178572,13.8
1,23020,1,nan,14616,2023-03-21 06:30:00+00:00,2023-03-23 05:30:00+00:00,80.4,18.80,91.2,181600000.0,14246,NaN,NaN,1676.160,6.56,169200,72.40,200179217,13.6
2,23021,1,nan,13170,2023-03-22 06:30:00+00:00,2023-03-24 05:30:00+00:00,66.4,16.16,86.4,248000000.0,17825,NaN,NaN,1928.496,8.08,169200,70.24,200181620,13.5
3,23022,1,nan,14614,2023-03-22 06:30:00+00:00,2023-03-24 05:30:00+00:00,85.6,18.48,83.2,229600000.0,12912,NaN,NaN,1782.800,5.92,169200,64.72,200182428,13.8
4,23023,1,nan,14615,2023-03-28 05:27:00+00:00,2023-03-30 08:00:00+00:00,77.6,17.12,74.4,132800000.0,17825,26.56,20.88,1861.840,2.96,181980,57.28,200182429,13.7


### cf_ino

In [1454]:
# Lote in cf but not in of
cf_of[cf_of['lote'].isin(ino['lote']) == False].count().max()

np.int64(26)

26 NAN values added for ino

In [1455]:
# Merge
cf_ino = cf.merge(ino, on=['lote'], how='left')
cf_ino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   lote                    152 non-null    object             
 1   orden_encadenado_cf     152 non-null    int64              
 2   lote_parental_cf        152 non-null    object             
 3   id_bio_x                152 non-null    object             
 4   f_h_inicio_cf           152 non-null    datetime64[ns, UTC]
 5   f_h_fin_cf              152 non-null    datetime64[ns, UTC]
 6   vol_ino_util_cf         147 non-null    float64            
 7   turb_inicio_cultivo_cf  152 non-null    float64            
 8   turb_fin_cultivo_cf     152 non-null    float64            
 9   viab_final_cultivo_cf   152 non-null    float64            
 10  id_centr                152 non-null    object             
 11  centr_1_turb_cf         148 non-null    float

### cf_pino 

In [1456]:
# Lote in cf but not in pino
cf_ino[cf_ino['lote'].isin(pino['lote']) == False].count().max()

np.int64(34)

In [1457]:
# Merge
cf_pino = cf.merge(pino, on=['lote'], how='left')
cf_pino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   lote                    152 non-null    object             
 1   orden_encadenado_cf     152 non-null    int64              
 2   lote_parental_cf        152 non-null    object             
 3   id_bio                  152 non-null    object             
 4   f_h_inicio_cf           152 non-null    datetime64[ns, UTC]
 5   f_h_fin_cf              152 non-null    datetime64[ns, UTC]
 6   vol_ino_util_cf         147 non-null    float64            
 7   turb_inicio_cultivo_cf  152 non-null    float64            
 8   turb_fin_cultivo_cf     152 non-null    float64            
 9   viab_final_cultivo_cf   152 non-null    float64            
 10  id_centr                152 non-null    object             
 11  centr_1_turb_cf         148 non-null    float

### MC

In [1458]:
# Max material unique values
mc['material_mc'].nunique()

13

In [1483]:
# Do all lotes have the same material?
mc.groupby('lote')['material_mc'].nunique().max()

np.int64(13)

We could add 13 mc columns

In [1459]:
# For each material, we can add a column
mc_copy = mc.copy()
for m in mc['material_mc'].unique():
    mc_copy[m + '_mc'] = 0
mc_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4920 entries, 0 to 4919
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   lote            4920 non-null   object             
 1   material_mc     4920 non-null   object             
 2   qty_mc          4920 non-null   float64            
 3   f_recepcion_mc  4920 non-null   datetime64[ns, UTC]
 4   f_traslado_mc   4920 non-null   datetime64[ns, UTC]
 5   duracion_mc     4920 non-null   int64              
 6   100001_mc       4920 non-null   int64              
 7   100002_mc       4920 non-null   int64              
 8   100003_mc       4920 non-null   int64              
 9   100004_mc       4920 non-null   int64              
 10  100005_mc       4920 non-null   int64              
 11  100006_mc       4920 non-null   int64              
 12  100007_mc       4920 non-null   int64              
 13  100008_mc       4920 non-null   i

In [1460]:
# For each if the material is in the row, we set the value to dur_mc
for i, row in mc.iterrows():
    for m in mc['material_mc'].unique():
        if row['material_mc'] == m:
            mc_copy.at[i, m + '_mc'] = row['duracion_mc']
mc_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4920 entries, 0 to 4919
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   lote            4920 non-null   object             
 1   material_mc     4920 non-null   object             
 2   qty_mc          4920 non-null   float64            
 3   f_recepcion_mc  4920 non-null   datetime64[ns, UTC]
 4   f_traslado_mc   4920 non-null   datetime64[ns, UTC]
 5   duracion_mc     4920 non-null   int64              
 6   100001_mc       4920 non-null   int64              
 7   100002_mc       4920 non-null   int64              
 8   100003_mc       4920 non-null   int64              
 9   100004_mc       4920 non-null   int64              
 10  100005_mc       4920 non-null   int64              
 11  100006_mc       4920 non-null   int64              
 12  100007_mc       4920 non-null   int64              
 13  100008_mc       4920 non-null   i

In [1461]:
mc_copy.head()

,lote,material_mc,qty_mc,f_recepcion_mc,f_traslado_mc,duracion_mc,100001_mc,100002_mc,100003_mc,100004_mc,100005_mc,100006_mc,100007_mc,100008_mc,100009_mc,100010_mc,100011_mc,100012_mc,100013_mc
0,23019,100001,72.0,2023-01-11 23:00:00+00:00,2023-03-21 23:00:00+00:00,5961600,5961600,0,0,0,0,0,0,0,0,0,0,0,0
1,23019,100002,72.0,2022-03-06 23:00:00+00:00,2023-03-14 23:00:00+00:00,32227200,0,32227200,0,0,0,0,0,0,0,0,0,0,0
2,23019,100003,4800.0,2021-08-26 22:00:00+00:00,2023-03-12 23:00:00+00:00,48643200,0,0,48643200,0,0,0,0,0,0,0,0,0,0
3,23019,100004,320.0,2021-11-24 23:00:00+00:00,2023-03-09 23:00:00+00:00,40608000,0,0,0,40608000,0,0,0,0,0,0,0,0,0
4,23019,100005,96.0,2022-11-03 23:00:00+00:00,2023-03-02 23:00:00+00:00,10281600,0,0,0,0,10281600,0,0,0,0,0,0,0,0


In [1462]:
# lote == 23019 and material == 100001
mc_copy[(mc_copy['lote'] == '23019') & (mc_copy['material_mc'] == '100001')]

,lote,material_mc,qty_mc,f_recepcion_mc,f_traslado_mc,duracion_mc,100001_mc,100002_mc,100003_mc,100004_mc,100005_mc,100006_mc,100007_mc,100008_mc,100009_mc,100010_mc,100011_mc,100012_mc,100013_mc
0,23019,100001,72.000,2023-01-11 23:00:00+00:00,2023-03-21 23:00:00+00:00,5961600,5961600,0,0,0,0,0,0,0,0,0,0,0,0
8,23019,100001,57.600,2023-01-11 23:00:00+00:00,2023-03-01 23:00:00+00:00,4233600,4233600,0,0,0,0,0,0,0,0,0,0,0,0
19,23019,100001,7.248,2023-01-11 23:00:00+00:00,2023-03-01 23:00:00+00:00,4233600,4233600,0,0,0,0,0,0,0,0,0,0,0,0


In [1463]:
# Is there only 1 value for each material and lote?
a = mc_copy.groupby(['lote', 'material_mc']).size().reset_index(name='count')
# Show more lines of a
a[a['count'] > 1]


,lote,material_mc,count
0,23019,100001,3
1,23019,100002,3
3,23019,100004,2
4,23019,100005,2
5,23019,100006,2
...,...,...,...
2294,24108,100004,2
2303,24109,100001,2
2304,24109,100002,2
2315,24110,100001,2


In [1464]:
mc_copy[(mc_copy['lote'] == '23019') & (mc_copy['material_mc'] == '100001')]

,lote,material_mc,qty_mc,f_recepcion_mc,f_traslado_mc,duracion_mc,100001_mc,100002_mc,100003_mc,100004_mc,100005_mc,100006_mc,100007_mc,100008_mc,100009_mc,100010_mc,100011_mc,100012_mc,100013_mc
0,23019,100001,72.000,2023-01-11 23:00:00+00:00,2023-03-21 23:00:00+00:00,5961600,5961600,0,0,0,0,0,0,0,0,0,0,0,0
8,23019,100001,57.600,2023-01-11 23:00:00+00:00,2023-03-01 23:00:00+00:00,4233600,4233600,0,0,0,0,0,0,0,0,0,0,0,0
19,23019,100001,7.248,2023-01-11 23:00:00+00:00,2023-03-01 23:00:00+00:00,4233600,4233600,0,0,0,0,0,0,0,0,0,0,0,0


In [1465]:
mc_copy[(mc_copy['lote'] == '24108') & (mc_copy['material_mc'] == '100001')]

,lote,material_mc,qty_mc,f_recepcion_mc,f_traslado_mc,duracion_mc,100001_mc,100002_mc,100003_mc,100004_mc,100005_mc,100006_mc,100007_mc,100008_mc,100009_mc,100010_mc,100011_mc,100012_mc,100013_mc
4898,24108,100001,57.808,2024-04-10 22:00:00+00:00,2024-06-19 22:00:00+00:00,6048000,6048000,0,0,0,0,0,0,0,0,0,0,0,0
4919,24108,100001,72.000,2024-04-10 22:00:00+00:00,2024-06-19 22:00:00+00:00,6048000,6048000,0,0,0,0,0,0,0,0,0,0,0,0


In [1466]:

# Perform average for each material and lote combination on duracion_mc
mc_copy = mc_copy.groupby(['lote', 'material_mc']).mean().reset_index()
mc_copy.head(30)

,lote,material_mc,qty_mc,f_recepcion_mc,f_traslado_mc,duracion_mc,100001_mc,100002_mc,100003_mc,100004_mc,100005_mc,100006_mc,100007_mc,100008_mc,100009_mc,100010_mc,100011_mc,100012_mc,100013_mc
0,23019,100001,45.616000,2023-01-11 23:00:00+00:00,2023-03-08 15:00:00+00:00,4809600.0,4809600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23019,100002,168.768000,2022-03-06 23:00:00+00:00,2023-03-14 23:00:00+00:00,32227200.0,0.0,32227200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,23019,100003,4800.000000,2021-08-26 22:00:00+00:00,2023-03-12 23:00:00+00:00,48643200.0,0.0,0.0,48643200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,23019,100004,200.000000,2021-11-24 23:00:00+00:00,2023-03-09 23:00:00+00:00,40608000.0,0.0,0.0,0.0,40608000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,23019,100005,54.040000,2022-11-03 23:00:00+00:00,2023-03-02 23:00:00+00:00,10281600.0,0.0,0.0,0.0,0.0,10281600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,23019,100006,100.032000,2022-03-06 23:00:00+00:00,2023-01-16 23:00:00+00:00,27302400.0,0.0,0.0,0.0,0.0,0.0,27302400.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,23019,100007,5220.000000,2023-02-08 23:00:00+00:00,2023-03-14 23:00:00+00:00,2937600.0,0.0,0.0,0.0,0.0,0.0,0.0,2937600.0,0.0,0.0,0.0,0.0,0.0,0.0
7,23019,100008,81.040000,2022-09-04 22:00:00+00:00,2023-03-01 23:00:00+00:00,15379200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15379200.0,0.0,0.0,0.0,0.0,0.0
8,23019,100009,4319.992000,2022-01-31 23:00:00+00:00,2023-03-13 11:00:00+00:00,35035200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35035200.0,0.0,0.0,0.0,0.0
9,23019,100010,54.040000,2022-06-01 22:00:00+00:00,2022-11-21 23:00:00+00:00,14947200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14947200.0,0.0,0.0,0.0


In [1467]:
# DUracion mc unique values
mc_copy['duracion_mc'].nunique()

506

In [1468]:
# Show first 10 rows, material and duration
mc_copy.head(10)

,lote,material_mc,qty_mc,f_recepcion_mc,f_traslado_mc,duracion_mc,100001_mc,100002_mc,100003_mc,100004_mc,100005_mc,100006_mc,100007_mc,100008_mc,100009_mc,100010_mc,100011_mc,100012_mc,100013_mc
0,23019,100001,45.616,2023-01-11 23:00:00+00:00,2023-03-08 15:00:00+00:00,4809600.0,4809600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23019,100002,168.768,2022-03-06 23:00:00+00:00,2023-03-14 23:00:00+00:00,32227200.0,0.0,32227200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,23019,100003,4800.000,2021-08-26 22:00:00+00:00,2023-03-12 23:00:00+00:00,48643200.0,0.0,0.0,48643200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,23019,100004,200.000,2021-11-24 23:00:00+00:00,2023-03-09 23:00:00+00:00,40608000.0,0.0,0.0,0.0,40608000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,23019,100005,54.040,2022-11-03 23:00:00+00:00,2023-03-02 23:00:00+00:00,10281600.0,0.0,0.0,0.0,0.0,10281600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,23019,100006,100.032,2022-03-06 23:00:00+00:00,2023-01-16 23:00:00+00:00,27302400.0,0.0,0.0,0.0,0.0,0.0,27302400.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,23019,100007,5220.000,2023-02-08 23:00:00+00:00,2023-03-14 23:00:00+00:00,2937600.0,0.0,0.0,0.0,0.0,0.0,0.0,2937600.0,0.0,0.0,0.0,0.0,0.0,0.0
7,23019,100008,81.040,2022-09-04 22:00:00+00:00,2023-03-01 23:00:00+00:00,15379200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15379200.0,0.0,0.0,0.0,0.0,0.0
8,23019,100009,4319.992,2022-01-31 23:00:00+00:00,2023-03-13 11:00:00+00:00,35035200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35035200.0,0.0,0.0,0.0,0.0
9,23019,100010,54.040,2022-06-01 22:00:00+00:00,2022-11-21 23:00:00+00:00,14947200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14947200.0,0.0,0.0,0.0


In [1469]:
# Drop material_mc, duracion_mc, f_recepcion_mc, f_entrega_mc, qty_mc
mc_copy.drop(columns=['material_mc', 'duracion_mc', 'f_recepcion_mc', 'f_traslado_mc', 'qty_mc'], inplace=True)

In [1470]:
mc_copy.head()


,lote,100001_mc,100002_mc,100003_mc,100004_mc,100005_mc,100006_mc,100007_mc,100008_mc,100009_mc,100010_mc,100011_mc,100012_mc,100013_mc
0,23019,4809600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23019,0.0,32227200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,23019,0.0,0.0,48643200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,23019,0.0,0.0,0.0,40608000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,23019,0.0,0.0,0.0,0.0,10281600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1471]:
# Number of unique lote in cf but not in mc
cf[cf['lote'].isin(mc['lote']) == False].count().max()

np.int64(22)

In [1472]:
# zeros to nan
mc_copy.replace(0, pd.NA, inplace=True)
mc_copy

,lote,100001_mc,100002_mc,100003_mc,100004_mc,100005_mc,100006_mc,100007_mc,100008_mc,100009_mc,100010_mc,100011_mc,100012_mc,100013_mc
0,23019,4809600.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,23019,<NA>,32227200.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,23019,<NA>,<NA>,48643200.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,23019,<NA>,<NA>,<NA>,40608000.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,23019,<NA>,<NA>,<NA>,<NA>,10281600.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2322,24110,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,9331200.0,<NA>,<NA>,<NA>,<NA>,<NA>
2323,24110,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,6825600.0,<NA>,<NA>,<NA>,<NA>
2324,24110,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,7344000.0,<NA>,<NA>,<NA>
2325,24110,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,11491200.0,<NA>,<NA>


In [1473]:
# Groupby lote and sum
mc_copy = mc_copy.groupby(['lote']).sum().reset_index()
mc_copy

,lote,100001_mc,100002_mc,100003_mc,100004_mc,100005_mc,100006_mc,100007_mc,100008_mc,100009_mc,100010_mc,100011_mc,100012_mc,100013_mc
0,23019,4809600.0,32227200.0,48643200.0,40608000.0,10281600.0,27302400.0,2937600.0,15379200.0,35035200.0,14947200.0,6134400.0,172800.0,0
1,23020,4809600.0,32227200.0,48643200.0,40809600.0,10281600.0,27302400.0,2937600.0,15379200.0,35035200.0,14947200.0,6134400.0,172800.0,0
2,23021,4809600.0,32227200.0,48643200.0,40910400.0,10281600.0,27302400.0,2937600.0,15379200.0,35035200.0,14947200.0,6134400.0,172800.0,0
3,23022,4809600.0,32227200.0,48643200.0,40910400.0,10281600.0,27302400.0,2937600.0,15379200.0,35035200.0,14947200.0,6134400.0,172800.0,0
4,23024,5961600.0,32227200.0,46310400.0,41385600.0,10281600.0,27302400.0,3628800.0,17323200.0,35337600.0,14947200.0,6134400.0,172800.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,24106,6048000.0,2462400.0,6825600.0,38016000.0,41990400.0,7776000.0,3974400.0,9331200.0,6393600.0,7344000.0,11491200.0,11145600.0,0
190,24107,6048000.0,2462400.0,6825600.0,38016000.0,41990400.0,7776000.0,3974400.0,9331200.0,6393600.0,7344000.0,11491200.0,11145600.0,0
191,24108,6048000.0,4147200.0,7344000.0,39182400.0,41990400.0,10713600.0,6566400.0,9590400.0,6825600.0,7344000.0,12009600.0,11145600.0,0
192,24109,6048000.0,4147200.0,6825600.0,38016000.0,41990400.0,10713600.0,3974400.0,9331200.0,6825600.0,7344000.0,11491200.0,11145600.0,0


,lote,material_mc,qty_mc,f_recepcion_mc,f_traslado_mc,duracion_mc


In [1474]:
# 100001_mc.value_counts
mc_copy['100001_mc'].value_counts().count()

np.int64(44)

In [1475]:
mc_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   lote       194 non-null    object
 1   100001_mc  194 non-null    object
 2   100002_mc  194 non-null    object
 3   100003_mc  194 non-null    object
 4   100004_mc  194 non-null    object
 5   100005_mc  194 non-null    object
 6   100006_mc  194 non-null    object
 7   100007_mc  194 non-null    object
 8   100008_mc  194 non-null    object
 9   100009_mc  194 non-null    object
 10  100010_mc  194 non-null    object
 11  100011_mc  194 non-null    object
 12  100012_mc  194 non-null    object
 13  100013_mc  194 non-null    object
dtypes: object(14)
memory usage: 21.3+ KB


In [1476]:
# Merge with cf
cf_mc = cf.merge(mc_copy, on='lote', how='left')
cf_mc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   lote                    152 non-null    object             
 1   orden_encadenado_cf     152 non-null    int64              
 2   lote_parental_cf        152 non-null    object             
 3   id_bio                  152 non-null    object             
 4   f_h_inicio_cf           152 non-null    datetime64[ns, UTC]
 5   f_h_fin_cf              152 non-null    datetime64[ns, UTC]
 6   vol_ino_util_cf         147 non-null    float64            
 7   turb_inicio_cultivo_cf  152 non-null    float64            
 8   turb_fin_cultivo_cf     152 non-null    float64            
 9   viab_final_cultivo_cf   152 non-null    float64            
 10  id_centr                152 non-null    object             
 11  centr_1_turb_cf         148 non-null    float

In [1484]:
# Merge everything
cf_mc_ino = cf_mc.merge(ino, on=['lote'], how='left')
cf_mc_ino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 39 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   lote                    152 non-null    object             
 1   orden_encadenado_cf     152 non-null    int64              
 2   lote_parental_cf        152 non-null    object             
 3   id_bio_x                152 non-null    object             
 4   f_h_inicio_cf           152 non-null    datetime64[ns, UTC]
 5   f_h_fin_cf              152 non-null    datetime64[ns, UTC]
 6   vol_ino_util_cf         147 non-null    float64            
 7   turb_inicio_cultivo_cf  152 non-null    float64            
 8   turb_fin_cultivo_cf     152 non-null    float64            
 9   viab_final_cultivo_cf   152 non-null    float64            
 10  id_centr                152 non-null    object             
 11  centr_1_turb_cf         148 non-null    float

In [1485]:
# Merge everything
cf_mc_ino_pino = cf_mc_ino.merge(pino, on=['lote'], how='left')

In [1487]:
cf_mc_ino_pino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 48 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   lote                    152 non-null    object             
 1   orden_encadenado_cf     152 non-null    int64              
 2   lote_parental_cf        152 non-null    object             
 3   id_bio_x                152 non-null    object             
 4   f_h_inicio_cf           152 non-null    datetime64[ns, UTC]
 5   f_h_fin_cf              152 non-null    datetime64[ns, UTC]
 6   vol_ino_util_cf         147 non-null    float64            
 7   turb_inicio_cultivo_cf  152 non-null    float64            
 8   turb_fin_cultivo_cf     152 non-null    float64            
 9   viab_final_cultivo_cf   152 non-null    float64            
 10  id_centr                152 non-null    object             
 11  centr_1_turb_cf         148 non-null    float

In [1488]:
# Merge of
cf_mc_ino_pino_of = cf_mc_ino_pino.merge(of, on=['lote'], how='left')


In [1489]:
cf_mc_ino_pino_of.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 50 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   lote                    152 non-null    object             
 1   orden_encadenado_cf     152 non-null    int64              
 2   lote_parental_cf        152 non-null    object             
 3   id_bio_x                152 non-null    object             
 4   f_h_inicio_cf           152 non-null    datetime64[ns, UTC]
 5   f_h_fin_cf              152 non-null    datetime64[ns, UTC]
 6   vol_ino_util_cf         147 non-null    float64            
 7   turb_inicio_cultivo_cf  152 non-null    float64            
 8   turb_fin_cultivo_cf     152 non-null    float64            
 9   viab_final_cultivo_cf   152 non-null    float64            
 10  id_centr                152 non-null    object             
 11  centr_1_turb_cf         148 non-null    float

In [1490]:
# to pkl
cf_mc_ino_pino_of.to_pickle(path + 'cf_mc_ino_pino_of.pkl')